# 理想 ADC 频谱分析示例

本笔记本按照 `spectrum_analysis_explained.ipynb` 的结构，模拟一个理想模数转换器：对连续时间正弦信号进行采样、量化，然后执行频谱分析并统计关键动态指标（SNR、ENOB、THD、SFDR 等）。


## 背景：理想 ADC 模型
理想 ADC 的处理流程包含：
1. **采样**：以固定频率 $F_s$ 将连续信号 $x(t)$ 离散化为 $x[n] = x(nT)$。
2. **量化**：将每个样本映射至有限集合的数字码字，理想量化器的误差 $e_q[n]$ 限定在半个 LSB 内。
3. **编码**：将量化结果转换为数字输出串。本示例聚焦前两步，并假设无时钟抖动、比较器延迟等非理想效应。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


## 步骤 1 · 采样与量化参数
- 采样频率 $F_s$
- 记录长度 $N$
- ADC 分辨率 $N_{\text{bit}}$
- 参考电压 $V_{\text{ref}}$

量化步长（LSB）满足 $\Delta = V_{\text{ref}} / 2^{N_{\text{bit}}}$，理想均匀量化器的输出电平位于区间中心。


In [ ]:
Fs = 2000               # 采样频率 (Hz)
Fin = 123.0             # 输入信号频率 (Hz)
L = 4096                # 采样点数
T = 1.0 / Fs
nbits = 12              # ADC 分辨率
vref = 1.8              # 全量程范围 (V)
lsb = vref / (2 ** nbits)
t = np.arange(L) * T


## 步骤 2 · 模拟连续时间信号
为便于分析，构造一个满量程的单音信号并叠加轻微噪声：
\[
x(t) = 0.45 V_{\text{ref}} \cdot \sin(2\pi F_{\text{in}} t) + w(t)
\]
其中噪声项 $w(t)$ 用零均值高斯噪声逼近。


In [ ]:
analog_signal = 0.45 * vref * np.sin(2 * np.pi * Fin * t)
analog_signal += 0.002 * vref * np.random.randn(L)

# 限幅在 ADC 量程内
min_input = -vref / 2
max_input = vref / 2
analog_signal = np.clip(analog_signal, min_input, max_input)


## 步骤 3 · 理想量化
对样本执行中点舍入量化：
\[
y[n] = \operatorname{clip}\left(\operatorname{round}\left(\frac{x[n]+V_{\text{ref}}/2}{\Delta}\right)\Delta - \frac{V_{\text{ref}}}{2}\right)
\]
得到的 $y[n]$ 即理想 ADC 输出对应的重构电压。


In [ ]:
levels = 2 ** nbits
raw_codes = np.round((analog_signal - min_input) / lsb)
raw_codes = np.clip(raw_codes, 0, levels - 1)
quantized_signal = raw_codes * lsb + min_input


## 步骤 4 · FFT 频谱
按单边幅度谱方式处理离散数据（与 `spectrum_analysis_explained.ipynb` 相同），以便评估量化后的频率特性。


In [ ]:
Y = np.fft.fft(quantized_signal)
P2 = np.abs(Y / L)
P1 = P2[: L // 2 + 1]
P1[1:-1] *= 2
f = Fs * np.arange(0, L // 2 + 1) / L


In [ ]:
peaks, _ = find_peaks(P1, prominence=0.2)
print("Detected dominant bins")
print("-" * 30)
if peaks.size:
    for idx in peaks[:5]:
        print(f"f = {f[idx]:7.2f} Hz | amplitude = {P1[idx]:.4f}")
else:
    print("No peaks above threshold.")


## 步骤 5 · 动态指标
沿用先前的功率定义，通过在频域屏蔽基波及其谐波附近若干 FFT 频点，得到噪声功率并推导出 SNR、ENOB、THD、SFDR 等指标。


In [ ]:
def compute_dynamic_metrics(P1, f, fs, peak_indices, harmonic_count=5, guard_bins=2):
    spectrum = np.asarray(P1)
    freq_axis = np.asarray(f)
    peak_indices = np.asarray(peak_indices)
    eps = 1e-20

    if spectrum.size < 2:
        raise ValueError("频谱长度不足以计算动态指标。")

    candidates = peak_indices if peak_indices.size else np.arange(1, spectrum.size)
    fundamental_idx = int(candidates[np.argmax(spectrum[candidates])])

    fundamental_freq = freq_axis[fundamental_idx]
    fundamental_amp = spectrum[fundamental_idx]
    fundamental_power = 0.5 * fundamental_amp ** 2

    harmonic_records = []
    harmonic_indices = []
    for order in range(2, harmonic_count + 1):
        target_freq = order * fundamental_freq
        if target_freq >= fs / 2:
            break
        idx = int(np.argmin(np.abs(freq_axis - target_freq)))
        harmonic_indices.append(idx)
        harmonic_records.append(
            {
                "order": order,
                "freq": freq_axis[idx],
                "amp": spectrum[idx],
                "power": 0.5 * spectrum[idx] ** 2,
            }
        )

    noise_mask = np.ones_like(spectrum, dtype=bool)

    def notch(mask, center, width):
        start = max(0, center - width)
        stop = min(mask.size, center + width + 1)
        mask[start:stop] = False

    notch(noise_mask, 0, guard_bins)  # DC
    notch(noise_mask, fundamental_idx, guard_bins)
    for idx in harmonic_indices:
        notch(noise_mask, idx, guard_bins)

    noise_power = np.sum(0.5 * np.square(spectrum[noise_mask]))
    harmonics_power = sum(record["power"] for record in harmonic_records) if harmonic_records else 0.0
    spur_indices = np.where(noise_mask)[0]
    spur_amplitude = spectrum[spur_indices].max() if spur_indices.size else 0.0

    snr_db = 10 * np.log10((fundamental_power + eps) / (noise_power + eps))
    enob = (snr_db - 1.76) / 6.02
    thd_db = (
        10 * np.log10((harmonics_power + eps) / (fundamental_power + eps))
        if harmonic_records
        else float("-inf")
    )
    sfdr_db = (
        20 * np.log10((fundamental_amp + eps) / (spur_amplitude + eps))
        if spur_amplitude > 0
        else float("inf")
    )

    return {
        "fundamental_freq": fundamental_freq,
        "fundamental_amp": fundamental_amp,
        "signal_power": fundamental_power,
        "noise_power": noise_power,
        "snr_db": snr_db,
        "enob": enob,
        "thd_db": thd_db,
        "sfdr_db": sfdr_db,
        "harmonics": harmonic_records,
    }


def format_db(value):
    if np.isposinf(value):
        return "+∞ dB"
    if np.isneginf(value):
        return "-∞ dB"
    return f"{value:.2f} dB"


metrics = compute_dynamic_metrics(P1, f, Fs, peaks, harmonic_count=5, guard_bins=2)

print(f"基波频率: {metrics['fundamental_freq']:.2f} Hz")
print(f"信号功率: {metrics['signal_power']:.4e}")
print(f"噪声功率: {metrics['noise_power']:.4e}")
print(f"SNR: {format_db(metrics['snr_db'])}")
print(f"ENOB: {metrics['enob']:.2f} bit")
print(f"THD: {format_db(metrics['thd_db'])}")
print(f"SFDR: {format_db(metrics['sfdr_db'])}")

if metrics["harmonics"]:
    print("\n谐波统计:")
    for record in metrics["harmonics"]:
        print(
            f"{record['order']} 次谐波 -> {record['freq']:.2f} Hz, "
            f"幅度 = {record['amp']:.4f}, 功率 = {record['power']:.3e}"
        )
else:
    print("\n未检测到可量化的谐波。")


## 步骤 6 · 可视化
第一张子图展示模拟输入与量化输出的差异，第二张子图给出单边幅度谱及检测到的峰值。


In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 1, 1)
plt.plot(t, analog_signal, label='Analog Input', linewidth=1.0)
plt.step(t, quantized_signal, where='mid', label='Quantized Output', linewidth=0.8)
plt.xlim(0, 10 / Fs)
plt.title('Time-Domain View of Ideal ADC')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(f, P1, label='Single-Sided Spectrum')
plt.plot(f[peaks], P1[peaks], 'x', color='red', markersize=8, label='Detected Peaks')
plt.title('Amplitude Spectrum of Quantized Signal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('|P1(f)|')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()


## 使用说明
1. 逐个运行本笔记本的单元，即可生成示例信号并自动计算动态指标。
2. 若要评估不同 ADC 分辨率或输入信号，只需修改“采样与量化参数”单元中的 `Fs`、`Fin`、`nbits`、`vref` 等变量。
3. `guard_bins` 可依据窗函数与量化噪声水平调整，以防能量泄漏影响噪声功率。
4. 若需要更真实的 ADC 行为，可在量化前添加失配、抖动或非均匀量化步骤，再重复同样的频谱分析。
